# **Project openmv**

### Enable GPUs for the notebook:

    - Navigate to Edit→Notebook Settings
    - select GPU from the Hardware Accelerator drop-down

Then run following section

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


To see the speedup by using a GPU form Google Colab see:
https://colab.research.google.com/notebooks/gpu.ipynb#scrollTo=oM_8ELnJq_wd

### Load image folder and insert model folder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!unzip -uq "/content/drive/My Drive/Embedded/Project/project-openmv/DataSet_1.zip"
#!unzip -uq "/content/drive/My Drive/project-openmv/DataSet_1.zip"
!unzip -uq "/content/drive/My Drive/Embedded/Project/project-openmv/DataSet_2.zip"

In [ ]:
# Insert directory containing the python scripts
import sys
sys.path.insert(0,"/content/drive/My Drive/Embedded/Project/project-openmv/PythonCode/CNNs")
#sys.path.insert(0,"/content/drive/My Drive/project-openmv/PythonCode/CNNs")

### Import needed libraries

In [ ]:
# Set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2 as cv
import os

#from lenet import LeNet
#from mobilenetv2 import MobileNetV2
#from squeezenet import SqueezeNet

from lenet_prep import LeNet
from mobilenetv2_prep import MobileNetV2
from squeezenet_prep import SqueezeNet


### Initialize variables and create tensors

In [ ]:
# Train datset path
#tp = "./DataSet_1/TrainSet"
tp = "./DataSet_2/TrainSet"

# Initialize the number of epochs, learning rate and batch size
EPOCHS = 20
INIT_LR = 1e-3
BS = 32

# Initialize data and labels
data = []
labels = []

In [ ]:
print("[INFO] loading images...")
# Grab the image paths and randomly shuffle them
train_paths = sorted(list(paths.list_images(tp)))
random.seed(42)
random.shuffle(train_paths)

# Cycle over the input train images
for train_path in train_paths:

  # Load the train image, preprocess it and store in the data array
  train_image = cv.imread(train_path) # read in rgb
    
  # train_image = cv.resize(train_image, (64, 64))
  train_image = img_to_array(train_image)
  data.append(train_image)

  # Extract the class label form the train image paths
  label = train_path.split(os.path.sep)[-2]

  if label == "Arrow0":
    label = 0
  elif label == "Arrow45":
    label = 1
  elif label == "Arrow90":
    label = 2
  elif label == "Arrow135":
    label = 3
  elif label == "Arrow180":
    label = 4
  elif label == "Arrow225":
    label = 5
  elif label == "Arrow270":
    label = 6
  else:
    label = 7
  labels.append(label)
print(labels[0:10])
print(np.array(data).shape)

[INFO] loading images...
[1, 4, 0, 1, 4, 6, 4, 6, 4, 0]
(15791, 64, 64, 3)


In [ ]:
# Scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print(len(data))
print(len(labels))

# Partition the data into training and testing splits using 75% of the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.25, random_state=42)
print(type(np.float32(trainX)))

15791
15791
<class 'numpy.ndarray'>


In [ ]:
# Convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=8)
testY = to_categorical(testY, num_classes=8)

# Construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=5, width_shift_range=0.1,height_shift_range=0.1, shear_range=0.2,
                         zoom_range=0.2,horizontal_flip=False, fill_mode="nearest")

### Initialize the model

In [ ]:
# Initialize the model
import tensorflow as tf
print("[INFO] compiling the model...")

### Select a model

#model = LeNet.build(width=64, height=64, depth=1, classes=8)
#model = MobileNetV2.build((64,64,1),8)
model = SqueezeNet.build(input_shape=(64,64,3), classes=8)

# Selects the appropriate optimizer
if model.name == "squeezenet":
  sgd = tf.keras.optimizers.Adam(lr=0.0001)
  model.compile(optimizer=sgd, loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
else:
  opt = Adam(lr=INIT_LR, decay=(INIT_LR / EPOCHS))
  model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

model.summary()

[INFO] compiling the model...
Model: "squeezenet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 31, 31, 64)   1792        input_2[0][0]                    
__________________________________________________________________________________________________
relu_conv1 (Activation)         (None, 31, 31, 64)   0           conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 15, 15, 64)   0           relu_conv1[0][0]                 
___________________________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


### Train the network

In [ ]:
# Train the network
print("[INFO] training the network...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS), validation_data=(testX, testY),steps_per_epoch=len(trainX) // BS, epochs=EPOCHS, verbose=1)

[INFO] training the network...


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
370/370 [==============================] - 17s 42ms/step - loss: 1.9958 - accuracy: 0.1872 - val_loss: 1.9207 - val_accuracy: 0.2090
Epoch 2/20
370/370 [==============================] - 15s 41ms/step - loss: 1.8038 - accuracy: 0.2834 - val_loss: 1.6883 - val_accuracy: 0.3663
Epoch 3/20
370/370 [==============================] - 16s 42ms/step - loss: 1.4571 - accuracy: 0.4485 - val_loss: 0.8269 - val_accuracy: 0.7138
Epoch 4/20
370/370 [==============================] - 15s 41ms/step - loss: 0.8330 - accuracy: 0.7001 - val_loss: 0.4797 - val_accuracy: 0.8440
Epoch 5/20
370/370 [==============================] - 15s 41ms/step - loss: 0.5969 - accuracy: 0.7838 - val_loss: 0.4205 - val_accuracy: 0.8422
Epoch 6/20
370/370 [==============================] - 15s 41ms/step - loss: 0.4422 - accuracy: 0.8410 - val_loss: 0.2062 - val_accuracy: 0.9428
Epoch 7/20
370/370 [==============================] - 15s 40ms/step - loss: 0.3464 - accuracy: 0.8821 - val_loss: 0.2373 - val_accuracy:

In [ ]:
# Save the model
print("[INFO] saving the network...")
model.save(model.name + ".h5")

[INFO] saving the network...


In [ ]:
# Save model history as png and as json file to be used in matlab

print("\n model history: ", H.history)

# Plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="center right")
sf = model.name + "_history.png"
plt.savefig(sf)
plt.show()

import json
a_file = open(model.name + "_history.json", "w")
json.dump(H.history, a_file)
a_file.close()



 model history:  {'loss': [1.9957998991012573, 1.8037528991699219, 1.4571478366851807, 0.8330432176589966, 0.5968717932701111, 0.44218239188194275, 0.34642109274864197, 0.33253511786460876, 0.27754300832748413, 0.22154733538627625, 0.19557493925094604, 0.20153166353702545, 0.1627376526594162, 0.15576137602329254, 0.16138778626918793, 0.12600941956043243, 0.11685818433761597, 0.11516251415014267, 0.11544007807970047, 0.11342471092939377], 'accuracy': [0.18719837069511414, 0.2833799123764038, 0.4484802186489105, 0.7001100778579712, 0.7837609052658081, 0.8409956693649292, 0.8820590972900391, 0.8860384225845337, 0.9044958353042603, 0.9250698685646057, 0.9338752031326294, 0.9315045475959778, 0.9442892074584961, 0.9479299187660217, 0.9464905858039856, 0.9605452418327332, 0.964101254940033, 0.9597832560539246, 0.9614765644073486, 0.9635086059570312], 'val_loss': [1.9207239151000977, 1.6882902383804321, 0.826867938041687, 0.4796561300754547, 0.42054155468940735, 0.20624211430549622, 0.2372604

## **Pruning**
https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras

In [ ]:
# Uncomment this section if you want to load an already trained .h5 model
from tensorflow.keras.models import load_model

print("[INFO] loading CNN...")
#model = load_model("/content/drive/MyDrive/Embedded/Project/project-openmv/Results/squeezenet/ndat/3channels/squeezenet.h5")


[INFO] loading CNN...


In [ ]:
# Install needed optimization toolkit
! pip install -q tensorflow-model-optimization

     |████████████████████████████████| 174kB 23.2MB/s 


Create model for pruning

In [ ]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after n epochs.
batch_size = BS
epochs = 20
validation_split = 0.1  # 10% of training set will be used for validation set. 

num_images = trainX.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.

# Select appropriate optimizer
if model_for_pruning.name == "squeezenet":
  sgd = tf.keras.optimizers.Adam(lr=0.0001)
  model_for_pruning.compile(optimizer=sgd, loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
else:
  model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()




/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py:2191: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Model: "squeezenet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
prune_low_magnitude_conv1 (Prun (None, 31, 31, 64)   3522        input_2[0][0]                    
__________________________________________________________________________________________________
prune_low_magnitude_relu_conv1  (None, 31, 31, 64)   1           prune_low_magnitude_conv1[0][0]  
__________________________________________________________________________________________________
prune_low_magnitude_pool1 (Prun (None, 15, 15, 64)   1           prune_low_magnitude_relu_conv1[0]
_________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Train the model

In [ ]:
import tempfile

logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

if model_for_pruning.name == "squeezenet":
  model_for_pruning.fit(trainX, trainY,
                      batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                      callbacks=callbacks)
else:
  model.fit_generator(aug.flow(trainX, trainY, batch_size=BS), 
                      validation_data=(testX, testY),steps_per_epoch=len(trainX) // BS, epochs=epochs, verbose=1)


Epoch 1/20
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
334/334 [==============================] - 22s 41ms/step - loss: 0.6969 - accuracy: 0.8203 - val_loss: 0.3510 - val_accuracy: 0.9004
Epoch 2/20
334/334 [==============================] - 12s 35ms/step - loss: 0.3838 - accuracy: 0.9062 - val_loss: 0.1873 - val_accuracy: 0.9654
Epoch 3/20
334/334 [==============================] - 12s 35ms/step - loss: 0.5199 - accuracy: 0.8686 - val_loss: 2.3802 - val_accuracy: 0.3316
Epoch 4/20
334/334 [==============================] - 12s 35ms/step - loss: 0.4493 - accuracy: 0.8767 - val_loss: 0.2599 - val_accuracy: 0.9578
Epoch 5/20
334/334 [==============================] - 12s 36ms/step - loss: 0.4282 - accuracy: 0.8921 - val_loss: 0.2089 - val_accuracy: 0.9679
Epoch 6/20
334/334 [==============================] - 12s 36ms/step - loss: 0.3962 - accuracy: 0.8915 - val_loss: 0.9254 - val_accuracy: 0.663

In [ ]:
# First, create a compressible model for TensorFlow

model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
model_for_export.save(model_for_pruning.name + 'Exp.h5', include_optimizer=False)

# Both tfmot.sparsity.keras.strip_pruning and applying a standard compression algorithm (e.g. via gzip) are necessary to see the compression benefits of pruning.

In [ ]:
# Then, create a compressible model for TFLite

converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

with open(model_for_export.name + 'Pruned.tflite', 'wb') as f:
  f.write(pruned_tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmpct7idu9b/assets


## **Quantization**

### Full integer quantization

In [ ]:
# Note: not working for MobileNetV2

import tensorflow as tf

def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(np.float32(trainX)).batch(1).take(100):
    yield [input_value]

#converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.target_spec.supported_types = [tf.int8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

converter.representative_dataset = representative_data_gen

tflite_model_quant_io = converter.convert()
with open(model_for_export.name + "FullIntQuant.tflite", 'wb') as f:
  f.write(tflite_model_quant_io)

INFO:tensorflow:Assets written to: /tmp/tmpmquegumr/assets


INFO:tensorflow:Assets written to: /tmp/tmpmquegumr/assets


### Integer with float fallback (using default float input/output)

In [ ]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(np.float32(trainX)).batch(1).take(100):
    yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

tflite_model_quant = converter.convert()
with open(model_for_export.name + "FloatFallbackQuant.tflite", 'wb') as f:
  f.write(tflite_model_quant)

INFO:tensorflow:Assets written to: /tmp/tmp9_ny3c97/assets


INFO:tensorflow:Assets written to: /tmp/tmp9_ny3c97/assets


##**Validation**

In [ ]:
# Define test images path
#testImg_folder = "./DataSet_1/ValidationSet"
testImg_folder = "./DataSet_2/ValidationSet"

Initialize data and labels

In [ ]:
# Initialize data and labels
print("[INFO] loading test images...")
test_images = []      # for .h5 models
test_labels = []

test_images_tf = []   # for .tflite models
test_labels_tf = []

# Grab the test image paths and randomly shuffle them
test_paths = sorted(list(paths.list_images(testImg_folder)))
random.seed(42)
random.shuffle(test_paths)

# Cycle over the input test images
for test_path in test_paths:

  # Load the test image, preprocess it and store in the test array
  test_image = cv.imread(test_path)   # read as rgb

  test_image = cv.resize(test_image, (64, 64))
  test_image = test_image.astype("float") / 255.0     # scale to [0,1]
  test_image = img_to_array(test_image)
  test_images_tf.append(test_image)
  test_image = np.expand_dims(test_image, axis=0)
  test_images.append(test_image)


  # Extract the class label form the train image paths
  label = test_path.split(os.path.sep)[-2]
  if label == "Arrow0":
    label = 0                       # for .h5 files
    label_tf = [1,0,0,0,0,0,0,0]    # for .tflite files
  elif label == "Arrow45":
    label = 1
    label_tf = [0,1,0,0,0,0,0,0]
  elif label == "Arrow90":
    label = 2
    label_tf = [0,0,1,0,0,0,0,0]
  elif label == "Arrow135":
    label = 3
    label_tf = [0,0,0,1,0,0,0,0]
  elif label == "Arrow180":
    label = 4
    label_tf = [0,0,0,0,1,0,0,0]
  elif label == "Arrow225":
    label = 5
    label_tf = [0,0,0,0,0,1,0,0]
  elif label == "Arrow270":
    label = 6
    label_tf = [0,0,0,0,0,0,1,0]
  else:
    label = 7
    label_tf = [0,0,0,0,0,0,0,1]
  test_labels.append(label)
  test_labels_tf.append(label_tf)

print(test_images[0].shape)
print(test_images_tf[0].shape)

[INFO] loading test images...
(1, 64, 64, 3)
(64, 64, 3)


### For .h5 files

Specify model and path to validation set folder

In [ ]:
# Define model path if you want to test an already existing trained .h5 model

#model_path = "/content/drive/MyDrive/Embedded/Project/project-openmv/Results/lenet/lenet.h5"

In [ ]:
# Load the trained CNN model

from tensorflow.keras.models import load_model

print("[INFO] loading CNN...")
model_test = model
#model_test = model_for_pruning
#model_test = model_for_export
#model_test = load_model(model_path)

[INFO] loading CNN...


Make predictions and compute the confusion matrix

In [ ]:
predictions = []

# cycle over the test images
print("[INFO] compute predictions...")
for j in range(0, len(test_images)):

  # Predict image
  prediction = model_test.predict(test_images[j])[0]
  predictions.append(np.argmax(prediction))

#print(len(predictions))
#print(len(labels))

from tensorflow import math as tfMath

confusionMatrix = tfMath.confusion_matrix(test_labels, predictions)
print(confusionMatrix)


[INFO] compute predictions...
tf.Tensor(
[[478   1   0   1   0   0   0   0]
 [  0 456   0   0   0   0   1   0]
 [  0   0 476   2   0   0   0   0]
 [  0   0   0 477   0   0   0   3]
 [  0   0   0   8 471   0   0   0]
 [  1   6   0   0   1 456   4   1]
 [  0   0   0   2   0   0 453   0]
 [  0   0   0   5   0   0   0 453]], shape=(8, 8), dtype=int32)


In [ ]:
# Save the confusion matrix
np.savetxt(model_test.name + "ConfMatr.txt", confusionMatrix, delimiter=',')

Compute the accuracy of the above selected model

In [ ]:
# models must be compiled to can perform the evaluate attribute
import tensorflow as tf
#   Select appropriate optimizer
if model_test.name == "squeezenet":
  sgd = tf.keras.optimizers.Adam(lr=0.0001)
  model_test.compile(optimizer=sgd, loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
else:
  model_test.compile(optimizer='adam',
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
_, model_accuracy = model_test.evaluate(
    np.array(test_images_tf), np.array(test_labels_tf), verbose=0)
    
print(model_accuracy)

0.9904153347015381


In [ ]:
# Save accuracy to file
with open(model_test.name + "Accuracy.txt", 'w') as f:
  f.write("%f" % model_accuracy)


### Evaluate Pruned Model


In [ ]:
# Load the trained CNN model

print("[INFO] loading CNN...")
model_test = pruned_tflite_model

[INFO] loading CNN...


In [ ]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on ever y image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images_tf):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

    #if i == 1000:
    #    break

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.

  correctPositives = 0
  ary = np.array([np.array(prediction_digits), np.array(test_labels)])

  for n in range(0,len(ary[0])):
    if ary[0][n] == ary[1][n]:
      correctPositives += 1

  return correctPositives/len(ary[0])

In [ ]:
interpreter = tf.lite.Interpreter(model_content=model_test)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('accuracy:', test_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.


accuracy: 0.9904153354632588


In [ ]:
# Save accuracy to file
with open("Pruned" + "Accuracy.txt", 'w') as f:
  f.write("%f" % test_accuracy)

#Evaluate Full Integer quantization

In [ ]:
# Load the trained CNN model

print("[INFO] loading CNN...")
model_test = tflite_model_quant_io

[INFO] loading CNN...


In [ ]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  input_details = interpreter.get_input_details()[0]
  
  # Inputs for the TFLite model must be uint8, so we quantize our input data
  scale, zero_point = input_details['quantization']
  
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images_tf):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.uint8(test_image / scale + zero_point)

    test_image = np.expand_dims(test_image, axis=0).astype(np.uint8)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

    #if i == 1000:
    #    break

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy

  correctPositives = 0
  ary = np.array([np.array(prediction_digits), np.array(test_labels)])

  for n in range(0,len(ary[0])):
    if ary[0][n] == ary[1][n]:
      correctPositives += 1

  print(np.array(prediction_digits))
  print(np.array(test_labels))

  return correctPositives/len(ary[0])

In [ ]:
interpreter = tf.lite.Interpreter(model_content=model_test)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('accuracy:', test_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.


[5 6 2 ... 0 0 7]
[5 6 2 ... 0 0 7]
accuracy: 0.9901490947816827


In [ ]:
# Save accuracy to file
with open("FullIntegerQuantization" + "Accuracy.txt", 'w') as f:
  f.write("%f" % test_accuracy)